# PKHG: free to use

# Killersudoku
### possible startnotebook by changing the conditions
with example from killersudoku.nl from 16 juni 2019

In [2]:
from mydateFP import getFPnameDate

In [4]:
# !cat mydateFP.py


# killersudoku.nl van 16 juni 2019
<img src="killersudoku.nl_16jun19.png"/>

# if ... only 3x9 part used
<img src="sudokumix5jun.jpg"/>

In [2]:
# show at most x solutions here x = 1
maxaantal = 43 #means show ONE or TWO solution if exists
from z3 import *
from mydateFP import getFPnameDate

#print inbetween results
tussenresultaat = True
fpname = getFPnameDate()
print("results will be saved in  (this working dir): ", fpname)
fp = open(fpname , "w")

#"""
# from an earlier killersudoku only the first row
# of 3x3 mini-sudoku used
# a 3x9 Sudoku where ONE number is given a ^ lower right corner
NulSodoku = [0,0,0] #sort of init
NulSodoku[0] = [0,0,0,0,0,0,0,0,0] #[0 for i in range(9)]
NulSodoku[1] = [0,0,0,0,0,0,0,0,0]
NulSodoku[2] = [0,0,0,0,0,0,0,0,6]
instance = NulSodoku
#"""
#instance = instPKHG = [ [0 for i in range(9)] for j in range(9)]


# 9x9 matrix of integer variables, zero based indexing!
X = [ [ Int("x_%s_%s" % (i, j)) for j in range(9) ]
      for i in range(9) ]

# each cell contains a value in {1, ..., 9}
cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9)
             for i in range(9) for j in range(9) ]
#PKHG>tothier
# each row contains a digit at most once
rows_c   = [ Distinct(X[i]) for i in range(9) ]

# each column contains a digit at most once
cols_c   = [ Distinct([ X[i][j] for i in range(9)]) for j in range(9)]
 
sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j]
                        for i in range(3) for j in range(3) ])
             for i0 in range(3) for j0 in range(3) ]
######## old
# rectangles in the 3x9 Sudoku are given:
#"""
c00_10 = [X[0][6] + X[0][7]  == 9,
          X[1][6] + X[1][7]  == 5,
          X[2][6] + X[2][7] == 13,
          #subtract the known 6 in last column gives:
          X[0][8] + X[1][8] == 18 - 6,
          X[2][8] == 6,
          X[0][0] + X[1][0] == 11,
          X[0][1] + X[1][1] == 9,
          X[0][2] + X[0][3] == 14,
          X[1][2] + X[2][2] == 9,
          X[0][4] + X[0][5] == 9,
          X[1][4] + X[2][4] == 5,
          
          X[1][5] + X[2][5] == 18 - 5,
         ]
#"""
    #X[][] + X[][] == 2,
    #X[][] + X[][] + X[][] == 3,
    #X[][] + X[][] + X[][] + X[][] == 4,
'''
#  see picture above ;-) 
allgivenInfo = [
    X[0][0] + X[1][0] == 8,
    X[0][1] + X[0][2] == 14,
    X[0][3] + X[0][4] == 15,
    X[0][5] + X[0][6] == 7,
    X[0][7] + X[0][8] == 7,
    X[1][1] + X[1][2] + X[1][3] == 8,
    X[1][4] + X[1][5] + X[1][6] == 16,
    X[1][7] + X[1][8] + X[2][8] + X[3][8] + X[4][8] == 31,
    X[2][0] + X[2][1] + X[2][2] + X[2][3] + X[3][3] == 23,
    X[2][4] + X[2][5] == 9,
    X[2][6] + X[2][7] + X[3][7] + X[3][6] == 30,
    X[3][0] + X[4][0] == 5,
    X[3][1] + X[3][2] == 11,
    X[3][4] + X[3][5] == 5,
    X[4][1] + X[5][1] == 17,
    X[4][2] + X[4][3] == 8,
    X[4][4] == 8,
    X[4][5] + X[4][6] == 10,
    X[4][7] + X[5][7] == 7,
    X[5][0] + X[6][0] + X[6][1] == 14,
    X[5][2] + X[5][3]  == 12,
    X[5][4] + X[5][5] + X[5][6] == 8,
    X[6][2] + X[6][3] == 11,
    X[6][4] +X[6][5] == 15,
    X[6][6] + X[6][7] == 11,
    X[7][0] + X[8][0] + X[8][1] == 20,
    X[7][1] + X[7][2] + X[8][2] + X[8][3] == 16,
    X[7][3] + X[7][4] + X[7][5] == 12,
    X[7][6] +X[8][6] + X[8][7] +X[8][8] == 25,
    X[8][4] + X[8][5] == 9
]
'''

#all conditions put together
# sudoku_c = cells_c + rows_c + cols_c + sq_c + allgivenInfo
sudoku_c = cells_c + rows_c + cols_c + sq_c + c00_10


# sudoku instance, we use '0' for empty cells
instance_c = [ If(instance[i][j] == 0,
                  True,
                  X[i][j] == instance[i][j])
               for i in range(3) for j in range(3) ]

allSolutions = []
s = Solver()
s.add(sudoku_c + instance_c) 

#while loop will stop if counter is > maxaantal, or 
#no new solutions anymore found
counter  = 0
while s.check() == sat:
    counter += 1
    fp.write("p" + str(counter) + " = ")
    m = s.model()

### for a partial upper three rows change
### for i in range(9) into  for i in range(3)

    #solution as one big int ;-)
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]
          for i in range(3) ]
    #     for i in range(9) ]


    t = ""
    for el in r:
    #print(el)
        res = [ "" + str(c) for c in el]
        tmp = "".join(res)
        t += tmp
    fp.write(t + "\n")
    
    #save solutions
    ns = int(t)  #t is build as str now a big integer
    allSolutions.append(ns)
    
    #print(len(allSolutions))
    #print(counter , t)
    if tussenresultaat:
        print_matrix(r)
    
    # the solution found will be not anymore allowed: Not(And(vals))
    vals = [X[i][j] == m.evaluate(X[i][j]) for j in range(9)
            for i in range(3)]
    s.add(Not(And(vals)))
    if counter > maxaantal:
        break
fp.close()
allSolasSet = set(allSolutions)
if counter:
    print("all different?" , len(allSolutions) == counter, "counter = ", counter)
else:
    print("no solution, check instance", instance)
                    

results will be saved in  (this working dir):  ./resultfiles/mixSodudoku0617_0913.py
[[4, 6, 5, 9, 8, 1, 2, 7, 3],
 [7, 3, 8, 5, 2, 6, 1, 4, 9],
 [9, 2, 1, 4, 3, 7, 8, 5, 6]]
[[4, 6, 5, 9, 8, 1, 7, 2, 3],
 [7, 3, 8, 5, 2, 6, 1, 4, 9],
 [9, 2, 1, 4, 3, 7, 5, 8, 6]]
[[4, 6, 5, 9, 8, 1, 7, 2, 3],
 [7, 3, 8, 5, 2, 6, 4, 1, 9],
 [9, 2, 1, 4, 3, 7, 5, 8, 6]]
[[4, 6, 5, 9, 8, 1, 7, 2, 3],
 [7, 3, 8, 5, 2, 6, 4, 1, 9],
 [9, 2, 1, 4, 3, 7, 8, 5, 6]]
[[4, 6, 5, 9, 8, 1, 7, 2, 3],
 [7, 3, 8, 5, 2, 6, 1, 4, 9],
 [9, 2, 1, 4, 3, 7, 8, 5, 6]]
[[4, 6, 5, 9, 8, 1, 7, 2, 3],
 [7, 3, 8, 5, 2, 6, 4, 1, 9],
 [2, 9, 1, 4, 3, 7, 5, 8, 6]]
[[4, 6, 5, 9, 8, 1, 7, 2, 3],
 [7, 3, 8, 5, 2, 6, 1, 4, 9],
 [2, 9, 1, 4, 3, 7, 5, 8, 6]]
[[4, 6, 5, 9, 8, 1, 2, 7, 3],
 [7, 3, 8, 5, 2, 6, 1, 4, 9],
 [2, 9, 1, 4, 3, 7, 5, 8, 6]]
[[4, 6, 5, 9, 8, 1, 2, 7, 3],
 [7, 3, 8, 5, 2, 6, 1, 4, 9],
 [9, 2, 1, 4, 3, 7, 5, 8, 6]]
[[4, 6, 5, 9, 8, 1, 2, 7, 3],
 [7, 3, 8, 5, 2, 6, 4, 1, 9],
 [9, 2, 1, 4, 3, 7, 5, 8, 6]]
[[4, 6, 5, 9, 1

In [7]:
!cat ./resultfiles/mixSodudoku0617_0907.py

p1 = 259876143641392578378145692165423789492687315783951264524769831817534926936218457
